# TODO
## Unified Emotion
- Remove samples with no-emotion class
- Convert multiple labels to multiple examples with different labels
- Come up with better assignment scheme for train/valid/test splits
- Drop "#SemST" from ssec sentences

## Go Emotions
- Get rid of print when loading (low priority)
- Include cases for manual tokenizer
- Convert multiple labels to multiple examples with different labels (check with Luuk)

## Manual Tokenizer
- check if works for go emotion
- incorporate special tokens into huggingface tokenizer

## Dataloaders
- loop Stratifiedloader for infinite sampling
- Rewrite train script to use correct dataloaders

In [8]:
import torch

from data.utils.data_loader import StratifiedLoader, AdaptiveNKShotLoader

# Datasets
## Unified Emotion

Data source download: https://drive.google.com/file/d/1y7yjshepNRPhnh-Qz5MTRbnopGn7KzUm/view?usp=sharing
Originally from: https://github.com/sarnthil/unify-emotion-datasets


Klinger, R. & Bostan, L. (2018, August). An analysis of annotated corpora for emotion classification in text. In Proceedings of the 27th International Conference on Computational Linguistics (pp. 2104-2119).

In [8]:
import pandas as pd
from data.unified_emotion import unified_emotion, unified_emotion_info

pd.DataFrame(unified_emotion_info())

ModuleNotFoundError: No module named 'jsonlines'

In [9]:
unified = unified_emotion("./data/datasets/unified-dataset.jsonl",\
    include=['crowdflower', 'dailydialog', 'electoraltweets', 'emoint', 'emotion-cause', 'grounded_emotions', 'ssec', 'tec'])

unified.prep()

NameError: name 'unified_emotion' is not defined

In [4]:
unified.lens

{'grounded_emotions': 2585,
 'ssec': 4868,
 'crowdflower': 40000,
 'dailydialog': 102979,
 'emotion-cause': 2414,
 'tec': 21051,
 'emoint': 7102,
 'electoraltweets': 4056}

In [5]:
for k in unified.lens.keys():
    dataset = unified.datasets[k]['train']
    trainloader = StratifiedLoader(dataset=dataset, device=torch.device('cpu'), k=1)
    labels, text, _, _ = next(trainloader)
    print(k)
    for lab, sent in zip(labels, text):
        print(lab, sent)
    print()

grounded_emotions
1 @realDonaldTrump @POTUS @IvankaTrump Mental health benefits removed from trumpcare. #trumpcare https://t.co/pE5a3YFh7Z
0 RT @Tackspayer: @IorettaIynch @HITEXECUTIVE I'd prefer an unemployed Constitutional scholar, Barack Obama. The karma would be deliciously sâ¦

ssec
0 @brandileighhhhh its called sexual coercion, and it is the same as rape. #RapeCulture #SemST
2 #Northwest #HeatWave continues. First time. Ever. My tomato plants have fruit. In JUNE! #Oregon #Organic #SemST
3 Make sure to make it to the Brew House in Pella, IA tomorrow @ 3 to meet with @HillaryClinton supporters! #SemST
1 The guy in the multicolored shirt looks chi as fuck. #SemST
6 We are what we are. Nothing more, nothing less. #spirituality #SemST
4 Pretend I'm a #tree and #save me. -babies everywhereyouthgen #SemST
5 Serious question for my atheist libertarians: How can rights exist without God? #ChristianLibertarian #SemST

crowdflower
5 @jeffparks Good morning, sir
6 Fell down the stairs at da

## GoEmotion

In [2]:
from data.go_emotions import go_emotions

go_emotion = go_emotions(first_label_only=True)
go_emotion.prep()

go_emotion.lens

No config specified, defaulting to: go_emotions/simplified
Reusing dataset go_emotions (/Users/xinyichen/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/b781b3f96f1b333b895ded30861c0d4a07d66e1cfbdfb89bc3fb4d5fc899aa27)
100%|██████████| 6/6 [00:00<00:00, 38.08ba/s]


{'go_emotions': 36491}

In [25]:
from transformers import AutoTokenizer
import torch.utils.data as data

dataset = go_emotions(first_label_only=True)
dataset.prep()
# dataset.lens
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')




No config specified, defaulting to: go_emotions/simplified
Reusing dataset go_emotions (/Users/xinyichen/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/b781b3f96f1b333b895ded30861c0d4a07d66e1cfbdfb89bc3fb4d5fc899aa27)
100%|██████████| 6/6 [00:00<00:00, 32.19ba/s]


In [28]:
len(dataset['go_emotions']['train'])

27

In [30]:
train_loader, val_loader = dataset.get_dataloader('go_emotions', k=8,tokenizer=tokenizer, shuffle=True)

NameError: name 'MetaStratifiedLoader' is not defined

In [29]:
# use normal data loader
#need to apply tokenizer to text
train_loader = data.DataLoader(dataset['go_emotions']['train'], batch_size=4,shuffle=False , num_workers=4)
len(train_loader)

7

In [32]:
tokenizer(dataset['go_emotions']['train']['text'],return_tensors='pt',padding=True)

IndexError: list index out of range

In [48]:
for k in go_emotion.lens.keys():
    dataset = go_emotion.datasets[k]['train']
    trainloader = StratifiedLoader(dataset=dataset, device=torch.device('cpu'), k=1)
    labels, text, _, _ = next(trainloader)
    print(k)
    for lab, sent in zip(labels, text):
        print(lab, sent)
    print()


go_emotions
2 I've said it before, but yuck, kissing that bloody mouth... Watch out, [NAME], you don't know where that's been.
14 Yeah but you're not anymore I'm afraid.
3 Yeah I've noticed that. I probably give off Aspie traits to NTs but Aspie like traits bug me
26 Wow the [NAME] replies appeared in less than ten minutes this time!
15 Haaaaa this was just on OZ the other night. Thanks.
8 I need help with corsets but otherwise would be mortified to need help for every day clothes.
0 That would be cool too!
6 Maybe it’s just too early but I can’t figure out what that’s supposed to be.
1 So we need a jewelry store and a tv show. Bet lol
5 Avoid. Without a contract he can kick you out at short notice any time And when you pay a deposit make sure it's protected.
12 2. I hate to say it but it’s the same thing I do and I hate myself for it
16 I don't see any shoes on his feet after he lands. Kid is fucking dead.
17 Oh my goodness it's like a happy little cloud! What breed is it? Either way,

# Custom Tokenizer
Here we define some rules for manually cleaning the imported data.
Given this is all internet sourced, it's strongly recommended to define something at least.
Current manual tokenizer will:
- Correct the text encodings
- Align contractions with BERT tokenizers
- Handles emojis (using emoji package) and twitter handles
- Deals with some edge cases where Spacy's tokenizer fails

In [14]:
from transformers import AutoTokenizer, AutoModel

from data.utils.tokenizer import manual_tokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer.additional_special_tokens = ["HTTPURL", "@USER"]

ModuleNotFoundError: No module named 'transformers'

The raw data

In [47]:
dataset = unified.datasets['grounded_emotions']['train']
trainloader = StratifiedLoader(dataset=dataset, device=torch.device('cpu'), k=8)

labels, text, _, _ = next(trainloader)
text

NameError: name 'unified' is not defined

The same, but now manually tokenized, sample

In [11]:
list(map(manual_tokenizer, text))

['i know you @USER HTTPURL',
 "HTTPURL how many more ways can # 45 show us he doesn ' t give a fuck about america ( ns ) ! he s doing all he can to hurt us ! !",
 'rt @USER : andy : can not say it better than gump ... stupid is as stupid does ! these cuts are # pennywiseandpoundfoolish they will cost ...',
 '@USER he has no clue . like all the eo s .',
 'rt @USER : photographer spencer amonwatvorakul talks about quitting his day job for his dream # standcreative2 HTTPURL ...',
 "my # tmc 🇺 🇸 :smiling_face_with_sunglasses: , u have been on point :check_mark_button: :fire: !exemplary comebacks brilliant original content ! doesn ' t get any better than that !! getting caught up :smiling_face:  🇺 🇸 .",
 'rt @USER : # oklahoma authorities get shortey . # gop state senator , former # trump coordinator faces child prostitution charges . https :/ ...',
 'rt @USER : @USER @USER @USER @USER they should take your medical license for violating your oath . " first do n ...',
 'that is truly sickeni

Can be easily slotted into the data loading process
Does quite a lot longer though...

In [12]:
# Use below if you additionally want to limit sentences to those that overlap well with BERT
# Not recommended for initial training 
#unified.prep(text_tokenizer=manual_tokenizer, text_tokenizer_kwargs={'bert_vocab': tokenizer.vocab.keys(), 'OOV_cutoff' :0.5, 'verbose':True})

unified.prep(text_tokenizer=manual_tokenizer)

Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.
Removed sentence for bad encoding.


In [13]:
print('\nExample data')
for k in unified.lens.keys():
    dataset = unified.datasets[k]['train']
    trainloader = StratifiedLoader(dataset=dataset, device=torch.device('cpu'), k=1)

    labels, text, _, _ = next(trainloader)
    print(k)

    label_map = {v: k for k, v in unified.label_map[k].items()}
    tokenized_texts = list(map(tokenizer.decode, tokenizer(text)['input_ids']))
    for txt, label in zip(tokenized_texts, labels):
        print(label_map[label], txt)
    print()


Example data
grounded_emotions
sadness [CLS] @ user what regulations caused harm to remington? thank you. [SEP]
joy [CLS] i am doing it!! @ user @ user # atmosphereplus # socool # nbavirginnomore # staplescenter... httpurl [SEP]

ssec
anger [CLS] ( it s just a night watcheasy to set up. staggered pickets ( google it ) work best ) # whoisburningblackchurches # semst [SEP]
fear [CLS] rosalind peterson addressing un on how aerosol spraying ( chemtrails ) is affecting agriculture. # notadebate # geoengineering # semst [SEP]
joy [CLS] coming from a female that was taken away the ability to have children. i still believe women should have the ability to choose. # semst [SEP]
disgust [CLS] i don't buy a dress that i can not completely zip up and fasten on my own. # semst [SEP]
trust [CLS] people who have been pregnant can be pro choice. people who can not have kids can be pro choice. people who have a uterus can be # semst [SEP]
sadness [CLS] pretend i am a # tree and # save me. - babies eve

In [14]:
#go_emotion.prep(text_tokenizer=manual_tokenizer)

# Sampling
## Dataset sampling

In [15]:
from data.utils.sampling import dataset_sampler

source_name = dataset_sampler(unified, sampling_method='sqrt')
source_name

'dailydialog'

## Dataloaders
Changed somewhat from last time. 

Now dataloaders must be generated manually using specific dataset (dict with labels as keys, lists of examples as values).

Samples from data and returns **both** the support and query.

Thus,

IN: dataset

OUT: support labels, support text, query labels, query text

If Huggingface tokenizer is passed, text is full model input (attention masks, token types, etc.)

Can be fed into model as,

```
model(**text)
```

### Stratified Sampling
Traditional N-way k-shot, balanced across classes.

Requires manually specifying k, which corresponds to batch size.

In [11]:
from collections import Counter

from data.utils.data_loader import StratifiedLoader, AdaptiveNKShotLoader

In [12]:
dataset = unified.datasets['ssec']['train']
trainloader = StratifiedLoader(dataset=dataset, device=torch.device('cpu'), k=16)
support_labels, support_text, query_labels, query_text = next(trainloader)

NameError: name 'unified' is not defined

In [18]:
Counter(support_labels)

Counter({0: 16, 2: 16, 3: 16, 1: 16, 6: 16, 4: 16, 5: 16})

In [19]:
Counter(query_labels)

Counter({0: 16, 2: 16, 3: 16, 1: 16, 6: 16, 4: 16, 5: 15})

In [20]:
#while True:
#    next(trainloader)

### Adaptive N-way k-shot
Dataloader with adaptive/stochastic N-way, k-shot batches.

Support set has random number of examples per class, although proportional to class size.

Query set is always balanced.

Not all classes are present if more than 5 classes are present in the dataset.

Algorithm taken from:

    Triantafillou et al. (2019). Meta-dataset: A dataset of datasets for learning to learn from few examples. arXiv preprint arXiv:1903.03096.

Steps are as follows:
    
1. Sample subset of classes (min 5, max all classes)
    
2. Define query set size (max 10 per class)
    
3. Define support set size (max 128 for all)
    
4. Fill support set with samples, stochastically proportional to support set size
    
5. Fill query set with remaining samples


In [21]:
dataset = unified.datasets['ssec']['train']
trainloader = AdaptiveNKShotLoader(dataset=dataset, device=torch.device('cpu'), max_support_size=128)
support_labels, support_text, query_labels, query_text = next(trainloader)

In [22]:
Counter(support_labels)

Counter({4: 3, 6: 3, 0: 84, 5: 2, 3: 25, 2: 8})

In [23]:
Counter(query_labels)

Counter({4: 10, 6: 10, 0: 10, 5: 10, 3: 10, 2: 10})